### Unified Code for Tiling

In [1]:
import os
import rasterio 
import numpy as np
from rasterio.windows import Window
from rasterio.transform import Affine

from rasterio.enums import Resampling
import albumentations as A
from rasterio.features import rasterize
import geopandas as gpd

In [2]:
# Directories

input_dop_rcn_folder = "data/dop_resampled_crs_Normalize"
output_tiles_folder = "Z:/BuildingDetection/dop_tilled"

os.makedirs(output_tiles_folder, exist_ok=True)

In [3]:
# Tile size (in pixels)
tile_size = 256

In [5]:
def save_tile(dataset, window, transform, output_path):
    """
    Save a raster tile to a GeoTIFF file.
    """
    profile = dataset.profile
    profile.update(
        width=window.width,
        height=window.height,
        transform=transform
    )
    with rasterio.open(output_path, "w", **profile) as dst:
        for i in range(1, dataset.count + 1): #write all bands
            dst.write(dataset.read(i, window=window), i)

# Loop through each raster in the input folder
for filename in os.listdir(input_dop_rcn_folder):
    if filename.endswith(".tif"):
        input_path = os.path.join(input_dop_rcn_folder, filename)

        with rasterio.open(input_path) as src:
            # Calculate the number of tiles along width and height
            n_tiles_x = (src.width + tile_size - 1) // tile_size
            n_tiles_y = (src.height + tile_size - 1) // tile_size

            for i in range(n_tiles_x):
                for j in range(n_tiles_y):
                    # Define the window for the current tile
                    window = Window(
                        col_off=i * tile_size,
                        row_off=j * tile_size,
                        width=tile_size,
                        height=tile_size
                    )

                    # Adjust window size for boundary tiles
                    window = window.intersection(Window(0, 0, src.width, src.height))

                    # Calculate the transform for the current tile
                    transform = src.window_transform(window)

                    # Define output path for the tile
                    tile_filename = f"{os.path.splitext(filename)[0]}_tile_{i}_{j}.tif"
                    output_path = os.path.join(output_tiles_folder, tile_filename)

                    # Save the tile
                    save_tile(src, window, transform, output_path)

        print(f"Tiled {input_path} into {output_folder}")

print("Tiling complete!")


KeyboardInterrupt: 